In [ ]:
%%time
# coding: utf-8
!pip install -r requrements.txt
import nltk
import numpy as np
from scipy.sparse import csr_matrix, save_npz, load_npz, lil_matrix
from sklearn.preprocessing import normalize
from nltk.stem.snowball import SnowballStemmer
from string import punctuation
from collections import OrderedDict
#np.set_printoptions(threshold=np.inf)
import os

In [ ]:
%%time
stemmer = SnowballStemmer("russian")
extended_punctuation = sorted(set(punctuation).union(set(['–', '…', '«', '»', '*'])))

#text = 'Хотя сам в первый раз минут тридцать трясся, все не мог понять, этот водоворот черный или же в нем есть три желтые песчинки, что мерещатся мне в который уже раз... уф...' *2
#text = 'a b c b a a'
corpusdir = os.path.expanduser("~") + '/book/'
files = os.listdir(corpusdir)
text_dict = {}
str_punkt = ''.join(extended_punctuation)
str_empty = ' ' * len(str_punkt)

def cleanup(txt):
    table = str.maketrans(str_punkt, str_empty)
    txt = txt.translate(table).lower()
    table_2 = str.maketrans('qwertyuiopasdfghjklzxcvbnm0123456789', str_empty)
    txt = txt.translate(table_2)
    return txt


for file in files:
    fullpath =  corpusdir + file
    print("reading from {}".format(fullpath))
    text = open( fullpath, 'r').read()
    text_dict[file] = cleanup(text)

In [ ]:
%%time
word_dict = {}
for k, v in text_dict.items():
    word_dict[k] = [stemmer.stem(word) for word in nltk.word_tokenize(v)]

In [ ]:
%%time
for k, v in word_dict.items():
    print(k)
    print(' '.join(v[:10]))

In [ ]:
%%time
#text.replace('-', ' ')
#words = [stemmer.stem(word) for word in nltk.word_tokenize(text.replace('-', ' ').lower()) if word not in extended_punctuation]
words = []
for i in word_dict.values():
    words = words + i
print(len(words))

In [ ]:
%%time
lemlist = sorted([x for x in (set(words))])
order = len(lemlist)
print(len(words), order)

In [ ]:
%%time
#M = np.zeros((order,order))
#M = csr_matrix((order, order))
M = lil_matrix((order, order))
print(type(M))
print(M.shape)
print(M[1,2])

In [ ]:
%%time
for n, m in word_dict.items():
    print("Working on " + n)
    for i, j in zip(m[:-1],m[1:]):
        x = lemlist.index(i)
        y = lemlist.index(j)
        #print(i,j,x,y)
        M[x, y] = M[x, y] + 1
#print("  " + " ".join(words))
#print("  " + " ".join(lemlist))
#print(M.shape)

In [ ]:
%%time
M_normalized = normalize(M, norm='l1', axis=1, copy=False)
#M.todence()

In [ ]:
%%time
save_npz('book_mm.npz', M_normalized)

In [ ]:
%%time
open('lemlist.txt', 'w').write('\n'.join(lemlist))